# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# read file
df = pd.read_csv(output_data_file)

# print data frame
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Norman Wells,20,CA,1611901852,67,65.2820,-126.8329,-36.40,3.44
1,Qaanaaq,0,GL,1611901852,62,77.4840,-69.3632,-14.01,6.15
2,Busselton,0,AU,1611901852,43,-33.6500,115.3333,84.99,3.00
3,Hilo,75,US,1611901852,64,19.7297,-155.0900,73.40,9.08
4,Pisco,0,PE,1611901853,78,-13.7000,-76.2167,69.80,9.22
...,...,...,...,...,...,...,...,...,...
546,Governador Valadares,2,BR,1611901991,91,-18.8511,-41.9494,67.32,0.60
547,Songyang,100,CN,1611901829,33,34.4553,113.0281,53.96,5.70
548,El Rosario,0,MX,1611901992,94,24.2556,-107.1828,62.01,3.00
549,Senno,100,RU,1611901992,93,59.5336,33.9167,28.99,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df = df.loc[(df["Max Temp"] >= 70) & (df["Max Temp"] <= 80) &
                    (df["Wind Speed"] <= 10) & (df["Cloudiness"] == 0)]

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,Port Alfred,0,ZA,1611901864,82,-33.5906,26.8910,71.01,5.01
139,Tura,0,IN,1611901682,34,25.5198,90.2201,76.68,3.94
160,Sur,0,OM,1611901905,31,22.5667,59.5289,74.08,5.21
175,Mar del Plata,0,AR,1611901909,73,-38.0023,-57.5575,71.01,6.91
195,San Isidro,0,PE,1611901913,83,-12.1167,-77.0500,72.00,8.05
236,Vila Velha,0,BR,1611901924,83,-20.3297,-40.2925,77.00,8.05
279,Tomatlán,0,MX,1611901938,71,19.9333,-105.2500,70.34,3.49
285,Kontagora,0,NG,1611901939,17,10.3999,5.4695,73.60,9.08
296,Pasni,0,PK,1611901941,23,25.2631,63.4710,70.74,5.53
333,Mon,0,IN,1611901946,41,26.7500,95.1000,72.52,4.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"] = ""

# create a params dict that will be updated with new city each iteration
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    lat = row['Lat']
    lng = row['Lng']

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
        
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
        
    try:
        # fill in data frame
        hotel_df.loc[index, 'Hotel Name'] = cities_lat_lng['results'][0]['name']
        
    except:
        print(f"Skip row {index}")

# Drop missing rows and print data frame
#hotel_df = hotel_df.drop([103,260])
#hotel_df

<ipython-input-8-3615bcf11094>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Skip row 333
Skip row 432


In [9]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

# Assign the marker layer to a variable
hotel_locations = hotel_df[['Lat', 'Lng']]
markers = gmaps.marker_layer(hotel_locations)

# Add the layer to the map
fig.add_layer(markers)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
symbol_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…